This script loads the Caltech101 dataset (stored as JPEG images in subfolders), and then uses a pre-trained VGG16 network to obtain neural code features (from the fc2 layer) for each images (see fc2_VGG16.ipynb for more explanation). Those features are stored in pickle files, along with the corresponding class.

The dataset consists of images divided into 101 classes, as well as an extra background/clutter class with random images. We will save one pickle file for all 101 classes, and another for the background set.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import os
import glob
import pickle
import numpy as np

In [3]:
# load pre-trained VGG16 from Keras
base_model = VGG16(weights="imagenet")
fc1_model = Model(inputs=base_model.input, outputs=base_model.get_layer("fc1").output)

# obtain Caltech101 filepaths, classes, and labels. Convert images to fc1 neural codes
path = os.path.join("data", "caltech101", "classes")
data_paths = [filepath for filepath in glob.glob(os.path.join(path, "**"), recursive=True) if filepath.lower().endswith(".jpg")]
feature_size = int(fc1_model.output.shape[1])  # = 4096 for the fc2-layer of VGG16
data_classes = os.listdir(path)
n_examples = len(data_paths)
data_fc1 = np.empty((n_examples, feature_size))
data_labels = np.empty(n_examples, dtype=int)
i = 0
for y, c in enumerate(data_classes):
    print("loading class {}: {}".format(y, c))
    class_path = os.path.join(path, c)
    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        img = image.load_img(image_path, target_size=(224, 224))
        array = image.img_to_array(img)
        x = np.expand_dims(array, axis=0)
        x = preprocess_input(x)
        fc1 = fc1_model.predict(x)
        data_fc1[i] = fc1
        data_labels[i] = y
        i += 1

# obtain the Caltech101 "BACKGROUND_Google" filepaths and fc2 neural codes.
bg_path = os.path.join("data", "caltech101", "BACKGROUND_Google")
bg_data_paths = [os.path.join(bg_path, filepath) for filepath in os.listdir(bg_path)]
n_bg_examples = len(bg_data_paths)
bg_data_fc1 = np.empty((n_bg_examples, feature_size))
for i, image_path in enumerate(bg_data_paths):
    img = image.load_img(image_path, target_size=(224, 224))
    array = image.img_to_array(img)
    x = np.expand_dims(array, axis=0)
    x = preprocess_input(x)
    fc1 = fc1_model.predict(x)
    bg_data_fc1[i] = fc1

# save neural codes, labels, paths, and list of classes to pickle file
with open(os.path.join("data","caltech101_VGG16_fc1.p"), "wb") as f:
    pickle.dump((data_fc1, data_labels, data_paths, data_classes), f)

# save neural codes & paths of BACKGROUND_Google class to pickle file
with open(os.path.join("data","caltech101_VGG16_fc1_bg.p"), "wb") as f:
    pickle.dump((bg_data_fc1, bg_data_paths), f)

loading class 0: accordion
loading class 1: airplanes
loading class 2: anchor
loading class 3: ant
loading class 4: barrel
loading class 5: bass
loading class 6: beaver
loading class 7: binocular
loading class 8: bonsai
loading class 9: brain
loading class 10: brontosaurus
loading class 11: buddha
loading class 12: butterfly
loading class 13: camera
loading class 14: cannon
loading class 15: car_side
loading class 16: ceiling_fan
loading class 17: cellphone
loading class 18: chair
loading class 19: chandelier
loading class 20: cougar_body
loading class 21: cougar_face
loading class 22: crab
loading class 23: crayfish
loading class 24: crocodile
loading class 25: crocodile_head
loading class 26: cup
loading class 27: dalmatian
loading class 28: dollar_bill
loading class 29: dolphin
loading class 30: dragonfly
loading class 31: electric_guitar
loading class 32: elephant
loading class 33: emu
loading class 34: euphonium
loading class 35: ewer
loading class 36: Faces
loading class 37: Face